**1. Exploration of data**

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
from datetime import date
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../input/recruit-restaurant-visitor-forecasting-data"]).decode("utf8"))

In [ ]:
weather_data = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/WeatherData.csv', parse_dates=['calendar_date'])
weather_data.columns = weather_data.columns.str.replace('area_name', 'station_id')
#weather_data = weather_data[weather_data['station_id'].notnull()]

anm = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/area_name_mapping.csv')
display(weather_data.describe())
print(len(weather_data.station_id.unique()), ", total weather records: ", len(weather_data))

In [ ]:
display(anm.head())
print(len(anm))
anm_station_ids = anm.station_id.unique()

In [ ]:
weather_station_data = weather_data[weather_data.station_id.isin(anm_station_ids)]
display(weather_station_data.station_id.unique())
print(len(weather_station_data.station_id.unique()), "total weather data records: ", len(weather_station_data))

In [ ]:
weather_station_data.describe()

In [ ]:
weather_station_data_filled = weather_station_data.interpolate()
weather_station_data_filled.describe()

In [ ]:

sample_result = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/sample_submission.csv')
features_to_train = sample_result.copy(deep=True)
display(features_to_train.head())
features_to_train['air_store_id'] = features_to_train.id.str.slice(0, 20)
features_to_train['visit_date'] = pd.to_datetime(features_to_train.id.str.slice(21, 31))
features_to_train.drop(['visitors', 'id'], axis = 1, inplace=True)
print(len(features_to_train.air_store_id.unique()))
print(len(features_to_train.visit_date.unique()))
display(features_to_train.head())


di = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/date_info.csv', parse_dates=['calendar_date'])
features_to_train = pd.merge(features_to_train, di, left_on = 'visit_date', right_on = 'calendar_date', how = 'left')
features_to_train.drop(['calendar_date'], axis = 1, inplace=True)

airstore = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/air_store_info.csv')
features_to_train = pd.merge(features_to_train,airstore,on=['air_store_id'])

display(features_to_train.head())
display(weather_station_data_filled.head())

features_to_train = pd.merge(features_to_train, anm, on='air_area_name')
features_to_train = pd.merge(features_to_train, weather_station_data_filled, left_on=['station_id','visit_date'], right_on=['station_id','calendar_date'])

result_data_stub = features_to_train.copy(deep=True)
features_to_train['year'] = features_to_train['visit_date'].dt.year
features_to_train['month'] = features_to_train['visit_date'].dt.month
features_to_train['day'] = features_to_train['visit_date'].dt.day

features_to_train.drop(['visit_date', 'calendar_date', 'total_snowfall', 'deepest_snowfall',
                  'avg_vapor_pressure', 'avg_humidity', 'avg_sea_pressure', 'avg_local_pressure',
                  'solar_radiation', 'cloud_cover', 'high_temperature','low_temperature',
                        'avg_temperature', 'avg_wind_speed', 'hours_sunlight',
                        'station_id', 'precipitation'], axis=1, inplace=True)

print(len(features_to_train))
display(features_to_train.head())

air_selected_store_ids = features_to_train.air_store_id.unique()
print(len(air_selected_store_ids))


In [ ]:
avd = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/air_visit_data.csv', parse_dates=['visit_date'])
di = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/date_info.csv', parse_dates=['calendar_date'])
display(avd.head(2))
display(di.head(2))

avd_di = pd.merge(avd, di, left_on = 'visit_date', right_on = 'calendar_date', how = 'left') 
avd_di.drop(['calendar_date'], axis = 1, inplace=True)

display(avd_di.head(2))
print("The AIR visitor data count",len(avd), ". The AIR visitor data with holiday info : ", len(avd_di), "records.")
print('Total AIR restaurents in AVD', len(avd.air_store_id.value_counts()))

In [ ]:
asi = pd.read_csv('../input/recruit-restaurant-visitor-forecasting-data/air_store_info.csv')
display(asi.head())

print( 'Total AIR restaurents: ', len(asi))
print('Total AIR restaurents in ASI', len(asi.air_store_id.value_counts()))

#air = pd.merge(airres,airstore,on=['air_store_id'])

In [ ]:
air = pd.merge(avd_di,asi,on=['air_store_id'])
#air.drop(['air_area_name'], axis = 1, inplace=True)
display(air.head(2))
print('Total AIR restaurents count: ', len(air.air_store_id.value_counts()))


In [ ]:
plt.rcParams['figure.figsize'] = 16, 8
plt.show()


In [ ]:
airdata = air
airdata['year'] = air['visit_date'].dt.year
airdata['month'] = air['visit_date'].dt.month
airdata['day'] = air['visit_date'].dt.day
#airdata = airdata.drop(['visit_date'], axis = 1)
airdata.head()

In [ ]:
print(len(airdata))
airdata_with_sid = pd.merge(airdata, anm, on='air_area_name')
print(len(airdata_with_sid))
airdata_with_sid.head()

In [ ]:
display(weather_station_data_filled.head(2))

In [ ]:
print(len(airdata_with_sid))
airdata_full = pd.merge(airdata_with_sid, weather_station_data_filled, left_on=['station_id','visit_date'], right_on=['station_id','calendar_date'])
print(len(airdata_full))

In [ ]:
display(airdata_full.head())
airdata_full.isnull().sum()

In [ ]:
airdata_full.dtypes

In [ ]:
#airdata_full.drop(['visit_date', 'calendar_date', 'total_snowfall', 'deepest_snowfall'], axis=1, inplace=True)
airdata_full.drop(['visit_date', 'calendar_date', 'total_snowfall', 'deepest_snowfall',
                  'avg_vapor_pressure', 'avg_humidity', 'avg_sea_pressure', 'avg_local_pressure',
                  'solar_radiation', 'cloud_cover', 'high_temperature','low_temperature',
                   'avg_temperature', 'avg_wind_speed', 'hours_sunlight',
                   'station_id','precipitation'], axis=1, inplace=True)

display(airdata_full.head())

In [ ]:
airdata_full.isnull().sum()

In [ ]:
adff = airdata_full.interpolate()
display(adff.head())
display(adff.describe())
display(airdata_full.describe())

In [ ]:
# Split the data into features and target label
print(len(adff.air_store_id.unique()))
airdata = adff[adff.air_store_id.isin(air_selected_store_ids)]
print(len(airdata.air_store_id.unique()))

visitors_data = airdata['visitors']
features_data = airdata.drop('visitors', axis = 1)

In [ ]:
print('Check the training data: ')
display(visitors_data.head())
display(features_data.head())

In [ ]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
#visitors	holiday_flg	latitude	longitude	year	month	day	precipitation	
#avg_temperature	hours_sunlight	avg_wind_speed	avg_vapor_pressure	avg_humidity	
#avg_sea_pressure	avg_local_pressure	solar_radiation	cloud_cover	high_temperature	
#low_temperature
#numerical = ['latitude', 'longitude', 'year', 'month', 'day', 'precipitation','avg_temperature', 
#             'hours_sunlight', 'avg_wind_speed', 'avg_vapor_pressure', 'avg_humidity', 'avg_sea_pressure',
#            'avg_local_pressure', 'solar_radiation', 'cloud_cover', 'high_temperature', 'low_temperature']
numerical = [ 'year', 'month', 'day', 'latitude', 'longitude']
features_minmax_transform = pd.DataFrame(data = features_data)
features_minmax_transform[numerical] = scaler.fit_transform(features_data[numerical])

# Show an example of a record with scaling applied
display(features_minmax_transform.head(n = 5))

air_selected_store_ids = features_minmax_transform['air_store_id'].unique()
print(len(air_selected_store_ids))
#print(air_selected_store_ids)

In [ ]:
print(len(features_data.air_store_id.unique()))
print(len(features_data.day_of_week.unique()))
print(len(features_data.air_genre_name.unique()))
print(len(features_data.air_area_name.unique()))
print(6)

In [ ]:
# One-hot encode the 'features_minmax_transform' data using pandas.get_dummies()
features_final = pd.get_dummies(features_minmax_transform)

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print(len(encoded),' total features after one-hot encoding.')

# Uncomment the following line to see the encoded feature names
#print(encoded)

In [ ]:
# Import train_test_split
from sklearn.cross_validation import train_test_split

# Split the 'features' and 'visitors_data' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    visitors_data, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has ",X_train.shape[0], " samples.")
print("Testing set has ",X_test.shape[0]," samples.")
print(X_train.shape, X_test.shape, len(y_train), len(y_train)/ 10, len(y_train)/100 )

In [ ]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(float(y_pred[i]) + 1) - math.log(float(y[i]) + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
# Import two metrics from sklearn - r2_score and explained_variance_score
from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    # Calculate the training time
    results['train_time'] = end-start
        
    # Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    # Calculate the total prediction time
    results['pred_time'] = end-start
            
    # Compute rmsle on the first 300 training samples which is y_train[:300]
    results['rmsle_train'] = explained_variance_score(y_train[:300], predictions_train)
        
    # Compute rmsle on test set using explained_variance_score()
    results['rmsle_test'] = explained_variance_score(y_test, predictions_test)
    
    # Compute F-score on the the first 300 training samples using fbeta_score()
    results['r2_train'] = r2_score(y_train[:300], predictions_train)
        
    # Compute F-score on the test set which is y_test
    results['r2_test'] = r2_score(y_test, predictions_test)
       
    # Success
    print(learner.__class__.__name__, " trained on ",sample_size," samples.")
        
    # Return the results
    return results

In [ ]:
import matplotlib.pyplot as pl
import matplotlib.patches as mpatches

def evaluate(results, rmsle, r2):
    """
    Visualization code to display results of various learners.
    
    inputs:
      - learners: a list of supervised learners
      - stats: a list of dictionaries of the statistic results from 'train_predict()'
      - rmsle: The score for the naive predictor
      - r2: The score for the naive predictor
    """
  
    # Create figure
    fig, ax = pl.subplots(2, 3, figsize = (11,7))

    # Constants
    bar_width = 0.3
    colors = ['#A00000','#00A0A0','#00A000']
    
    # Super loop to plot four panels of data
    for k, learner in enumerate(results.keys()):
        for j, metric in enumerate(['train_time', 'rmsle_train', 'r2_train', 'pred_time', 'rmsle_test', 'r2_test']):
            for i in np.arange(3):
                
                # Creative plot code
                ax[int(j/3), int(j%3)].bar(i+k*bar_width, results[learner][i][metric], width = bar_width, color = colors[k])
                ax[int(j/3), int(j%3)].set_xticks([0.45, 1.45, 2.45])
                ax[int(j/3), int(j%3)].set_xticklabels(["1%", "10%", "100%"])
                ax[int(j/3), int(j%3)].set_xlabel("Training Set Size")
                ax[int(j/3), int(j%3)].set_xlim((-0.1, 3.0))
    
    # Add unique y-labels
    ax[0, 0].set_ylabel("Time (in seconds)")
    ax[0, 1].set_ylabel("rmsle")
    ax[0, 2].set_ylabel("R2-score")
    ax[1, 0].set_ylabel("Time (in seconds)")
    ax[1, 1].set_ylabel("rmsle")
    ax[1, 2].set_ylabel("R2-score")
    
    # Add titles
    ax[0, 0].set_title("Model Training")
    ax[0, 1].set_title("rmsle on Training Subset")
    ax[0, 2].set_title("R2-score on Training Subset")
    ax[1, 0].set_title("Model Predicting")
    ax[1, 1].set_title("rmsle on Testing Set")
    ax[1, 2].set_title("R2-score on Testing Set")
    
    # Add horizontal lines for naive predictors
    ax[0, 1].axhline(y = rmsle, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    ax[1, 1].axhline(y = rmsle, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    ax[0, 2].axhline(y = r2, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    ax[1, 2].axhline(y = r2, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    
    # Set y-limits for score panels
    ax[0, 1].set_ylim((0, 1))
    ax[0, 2].set_ylim((0, 1))
    ax[1, 1].set_ylim((0, 1))
    ax[1, 2].set_ylim((0, 1))

    # Create patches for the legend
    patches = []
    for i, learner in enumerate(results.keys()):
        patches.append(mpatches.Patch(color = colors[i], label = learner))
    pl.legend(handles = patches, bbox_to_anchor = (-.80, 2.53), \
               loc = 'upper center', borderaxespad = 0., ncol = 3, fontsize = 'x-large')
    
    # Aesthetics
    pl.suptitle("Performance Metrics for Three Supervised Learning Models", fontsize = 16, y = 1.10)
    pl.tight_layout()
    pl.show()

In [ ]:
import math
from sklearn.metrics import mean_squared_log_error

from sklearn.metrics import r2_score 

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

In [ ]:
from sklearn import linear_model, tree, svm, ensemble
from time import time

# Initialize the three models
#clf_A = tree.DecisionTreeRegressor(random_state = 5, max_depth=50, max_features='auto') # 0.41
#clf_B = AdaBoostRegressor(random_state = 5) #0.
#clf_C = BayesianRidge(n_iter = 300, tol = 0.005) #0.469
clf_A = linear_model.LinearRegression()
clf_B = linear_model.Ridge()
clf_C = linear_model.Lasso()
clf_D = linear_model.ElasticNet()
clf_E = linear_model.BayesianRidge()
clf_F = linear_model.RANSACRegressor()
clf_G = svm.SVR()
clf_H = ensemble.GradientBoostingRegressor()
clf_I = tree.DecisionTreeRegressor()
clf_K = ensemble.RandomForestRegressor()


# Calculate the number of samples for 1%, 10%, and 100% of the training data
# HINT: samples_100 is the entire training set i.e. len(y_train)
# HINT: samples_10 is 10% of samples_100
# HINT: samples_1 is 1% of samples_100
samples_100 = len(y_train)
samples_10 = samples_100/10
samples_1 = samples_100/100

# Collect results on the learners
results = {}
for clf in [clf_E, clf_K]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([int(samples_1), int(samples_10), int(samples_100)]):
        results[clf_name][i] = train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
evaluate(results, 0.0, 0.0)
print(results)

In [ ]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = [ 'year', 'month', 'day', 'latitude', 'longitude']

print('length of features_to_train: ', len(features_to_train))
features_td_minmax_transform = pd.DataFrame(data = features_to_train)
features_td_minmax_transform[numerical] = scaler.fit_transform(features_to_train[numerical])

#print(features_td_minmax_transform.visit_date.value_counts())
# Show an example of a record with scaling applied
display(features_td_minmax_transform.head(n = 5))

#print(features_td_minmax_transform.air_store_id.value_counts())
print(len(features_td_minmax_transform))

In [ ]:
print(len(features_to_train.air_store_id.unique()))
print(len(features_to_train.day_of_week.unique()))
print(len(features_to_train.air_genre_name.unique()))
print(len(features_to_train.air_area_name.unique()))
print(6)

In [ ]:
features_td_final = pd.get_dummies(features_td_minmax_transform)

# Print the number of features after one-hot encoding
encoded_td = list(features_td_final.columns)
print(len(encoded_td),' total features after one-hot encoding.')

# Uncomment the following line to see the encoded feature names
print(encoded_td)

In [ ]:
print(len(features_td_final))
result = clf_K.predict(features_td_final)
print(result)

In [ ]:
result_data_stub.head()

In [ ]:
result_data = result_data_stub[result_data_stub.air_store_id.isin(air_selected_store_ids)]
result_data['visitors'] = pd.Series(result, index=result_data.index)
result_data['id'] = result_data.air_store_id + result_data.visit_date.dt.strftime('_%Y-%m-%d')

print(len(result_data.id.unique()))
print(len(result_data))
print(len(result))
display(result_data)

In [ ]:
result_to_save = result_data[['id', 'visitors']]

display(result_to_save.head())

print(result_to_save.dtypes)
print(len(result_to_save[result_to_save.visitors < 0]))
result_to_save.loc[~(result_to_save['visitors'] > 0), 'visitors'] = 0
print(len(result_to_save[result_to_save.visitors < 0]))

display(result_to_save.head())

In [ ]:
print(len(result_to_save))

In [ ]:
result_to_save.to_csv('csv_to_submit.csv', index = False)